# Lab 9 - Community Detection Algorithms II

###  Peter Kairouz and Pramod Viswanath

In this lab, you will learn how to design **efficient community detection** algorithms to cluster a given social network into non-overlapping communities. 

After filling this notebook and running all the cells, rename the file **lab9.ipynb** to **firstname_lastname_lab9.ipynb**, include your **well commented** code, and submit it by email. Avoid unneeded steps/computations and make sure your code runs before submitting it. Grading is based on your submission which is due at **4 p.m. March 30, 2016**. Your grade will be deducted 20 points for each day after the due date (late penalty).

**This lab is partially adapted from UC Berkeley's EE126 by Prof. Kannan Ramchandran.**

----

## Functions needed from previous Lab 8

In [3]:
from random import random
def G(n,p):
    graph = [] 
    for i in xrange(n):
        graph.append((i,i))
    # in this lab, we describe a graph as a list of tuples enumerating all edges - node names can be numbers.
    for i in xrange (0,n):
        for e in xrange(i+1,n): #Go through all pairs of nodes
            if (p > random() ): #if the probability of connection is greater than the generated probablity
                graph.append((i,e)) #connect the nodes
    return graph

def find_highest(graph):
    temp = []
    for start, end in graph:
        temp.append(start)
        temp.append(end)
    return max(temp)

def adjacency_list(graph):
    """
    Takes in the current representation of the graph, outputs an equivalent
    adjacency list
    Example: graph = [(0,1), (1,2), (2,0)] --> adjacency = [ [1, 2], [0, 2], [0, 1]]
    """
    highest = find_highest(graph)                      #find the highest numbered node in the graph
    adjacency = [[] for x in xrange(highest + 1 )]     #create an adjacency list of that length
#    print len(adjacency)                               #Debug Code
    for start,end in graph:
        adjacency[start].append(end)                  #append to both nodes the other node
        adjacency[end].append(start)
    for x in xrange(0,highest + 1):
        adjacency[x] = list(set(sorted(adjacency[x])))          #sort each list in adjacency so we have a nice clean lowest to highest
        adjacency[x].remove(x)
    return adjacency                               #and return


def SBM(n,p,q):
    """
    Let the first n/2 nodes be part of community A and 
    the second n/2 part of community B.
    """
    assert(n % 2 == 0)
    mid = int(n/2)
    graph = []
    for i in xrange(n):
        graph.append((i,i))
        
    # create community A
    # your code goes here
    A = G(mid,p)                 
    graph.extend(A) #add it to the graph
    
    # create community B  
    # your code goes here
    B = G(mid,p) #create a second community of the same size
    i = 0        
    for start,end in B:
        B[i] = (start + mid,end + mid) #we need to increment all it's nodes by mid
        i += 1
    graph.extend(B) #and add it to the graph

    # form connections between communities
    for i in xrange(mid):
        for j in xrange(mid, n):
            if rnd.random() < q:
                graph.append( (i, j) )
    return graph

#from numpy import *
import numpy as np

def prob_recovery(L, n, alpha, beta):
    mid = int(n/2)
    ground_truth1 = tuple(np.arange(mid)) # community A
    ground_truth2 = tuple(np.arange(mid, n)) # community B
    p = (alpha)*np.log(n) / n
    q = (beta)*np.log(n) / n
    num_correct = 0
    
    # your code goes here
    
    # do the following L times
    for i in xrange(0,L):
    # generate an SBM graph
        graph = SBM(n,p,q)
    # use min_bisection(graph) to find the 2 communities in the randomly generated graph
        A,B = min_bisection(graph)
    # compare the communities returned by min_bisection to the true communities (community A and B)
        if A==ground_truth1 and B==ground_truth2:
            # if the match increment num_correct
            num_correct +=1


    
    return float(num_correct/L)


----

## Problem 1: Semidefinite Programming for Community Detection (50 pts)

In the previous lab, we used the min-bisection algorithm to reconstruct the two communities. As we saw in one of the previous questions, min-bisection is extremely inefficient. In this section, we will develop a more efficient algorithm for the community detection problem. We will use $G(V, E)$ to denote the undirected graph that we observe, where $V$ is the set of nodes $(|V|=n)$, and $E$ is the set of edges.

First, let's consider an intuitive algorithm to solve the community detection problem. As we have seen, the goal of community detection is to separate the nodes into two communities, such that the number of edges within the same community is as large as possible and the number of edges between two communities is as small as possible. To achieve this goal, we consider the "score" of a particular separation. For an edge within a community, we get one point; for an edge between two communities, we get minus one point. We want to maximize the score over all possible separations. We identify a choice of communities by a vector $x\in\mathbb{R}^n$ with $\pm1$ entries such that $x_i$ will be $+1$ if node $i$ is in one community and $-1$ if it is in the other. We also define $A$ as the $n\times n$ matrix with zero diagonal whose non diagonal entries are given by
$$
A_{ij}=\begin{cases}
1 & \text{if }(i,j)\in E\\
-1 & \text{if }(i,j)\notin E
\end{cases}
$$
Then we can show that, maximizing the score is equivalent to the following optimization problem (think about the reason by yourself):
\begin{align}
\max &~~x^TAx \\
s.t. &~~x_i=\pm1.
\end{align}
However, since this optimization problem is combinatorial and hard to solve, we need to relax the constraint that $x_i$ has to be $\pm1$.

Let's look at the objective of the optimization problem:
$x^TAx$. According to knowledge in linear algebra, we know that $x^TAx=\text{Tr}(x^TAx)=\text{Tr}(Axx^T)$. Here, "Tr" denotes the trace of a square matrix, i.e., the sum of all the elements on the diagonal. We can see that $x^TAx=\text{Tr}(x^TAx)$ is obvious because the trace of a scalar is still itself; and $\text{Tr}(x^TAx)=\text{Tr}(Axx^T)$ is because of the fact that $\text{Tr}(AB)=\text{Tr}(BA)$. If we denote the rank-one matrix $xx^T$ by $X$, then the previous optimization problem is equivalent to:
\begin{align}
\max &~~\text{Tr}(AX) \\
s.t. &~~X=xx^T\text{ and }x_i=\pm1.
\end{align}

Since this problem is still hard to solve, we need to relax the constraints on $X$. As we can see, the diagonal elements of $X$ are all 1. Further, we can see that $X$ is positive semidefinite.
(A matrix $D\in\mathbb{R}^{n\times n}$ is called a positive semidefinite matrix if and only if for any vector $u\in\mathbb{R}^n$, there is $u^TDu\ge 0$). 
An optimization problem with linear objective functions and matrix variables which are constrained to be positive semidefinite is called a semidefinite program (SDP). SDPs are convex optimization problems, and therefore, the global minimum can be found in polynomial time. Therefore, instead of solving the combinatorial optimization problem, we solve the following SDP problem:
\begin{align*}
\max &~~\text{Tr}(AX)\\
s.t. &~~X_{ii}=1\\
&X\succeq 0,
\end{align*}
and hope the the relaxed optimization problem can give us the same answer as the original problem. It is proved that if $\alpha$ and $\beta$ satisfy some conditions, the solution to the SDP problem $X^*$ will be the outer product of the solution to the combinatorial optimization problem $x^*$, i.e., $X^*=x^*x^{*T}$. We will use the CVX package for Python to solve this SDP:

http://cvxopt.org/.

Install CVX in your computer and read the instructions on solving SDP using CVX:

http://cvxopt.org/userguide/coneprog.html#semidefinite-programming.

Specifically, we will solve the dual SDP problem. We will use different data structures from the previous parts in order to use CVX. Therefore, we define some new functions which are useful in this part.

### Helper Functions for Semi-Definite Programming

In [4]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

def generate_sbm(n, alpha, beta):
    """
    Generate the A matrix for an SBM.
    inputs:  n: total number of nodes, 
             alpha: parameter alpha corresponding to the in-cluster connection probability
             beta: parameter beta corresponding to the cross-cluster connection probability
    outputs: A: the "A" matrix for the SBM. A(i,i)=0 for all i; A(i,j) = 1 if (i,j) is an edge; A(i,j)=-1 otherwise.
             truth: the ground truth of the two clusters, represented with +/- 1
    both A and truth are in the CVX matrix data structure 
    """
    assert(n % 2 == 0)
    mid = int(n/2)
    # generate parameters
    p = alpha*np.log(n)/n
    if p > 1.0:
        p = 1.0
    q = beta*np.log(n)/n
    if q > 1.0:
        q = 1.0
    # generate A matrix
    A = np.zeros([n, n])
    A[0:mid, mid:n] = np.random.binomial(1, q, (mid, mid))
    for i in range(mid):
        for j in range(i+1, mid):
            A[i, j] = np.random.binomial(1, p)
    for i in range(mid, n):
        for j in range(i+1, n):
            A[i, j] = np.random.binomial(1, p)
    A = A+np.transpose(A)
    A = (A-0.5)*2
    for i in range(n):
        A[i, i] = 0
    # randomly permute the rows and columns
    perm = np.random.permutation(n)
    A = A[:, perm]
    A = A[perm, :]
    # find the ground truth
    argperm = np.argsort(perm)
    truth = np.zeros([n, 1])
    truth[argperm[0:mid], 0] = 1
    truth[argperm[mid:n], 0] = -1
    # return A and truth
    return matrix(A), matrix(truth)

def is_correct(sol, truth):
    """
    Checks whether the reconstruction found by SDP is correct.
    inputs:  sol: the solution X^* found by SDP in CVX matrix data structure
             truth: ground truth x^*, a column vector in CVX matrix data structure
    outputs: 1 if reconstruction is correct; 0 otherwise
    """
    # set a threshold for the difference between elements of X^* and x^*X^{*T}
    th = 1e-4
    difference = abs(sol-truth*np.transpose(truth))
    if difference.max() < th:
        # exact recovery
        return 1
    else:
        # wrong recovery
        return 0

def recon_prob_sdp(n, alpha, beta):
    """
    Find the probability of successful reconstruction given the parameters
    inputs:  n: total number of nodes, 
             alpha: parameter alpha corresponding to the in-cluster connection probability
             beta: parameter beta corresponding to the cross-cluster connection probability
    outputs: the simulated probability of successful reconstruction
    """
    assert(n % 2 == 0)
    num_tests = 50
    num_success = 0.0
    for t in range(num_tests):
        result = generate_sbm(n, alpha, beta)
        A = result[0]
        truth = result[1]

        # Set parameters for the SDP
        c = matrix(-1., (n, 1))
        h = [-A]
        G1 = np.zeros([n*n, n])
        for i in range(n):
            G1[i+n*i, i] = 1
        G = [matrix(G1)]
        sol = solvers.sdp(c, Gs=G, hs=h)
        sol = sol['zs'][0]
        if is_correct(sol, truth) == 1:
            num_success = num_success + 1
    return float(num_success)/float(num_tests)

### <font color=blue>Use the above helper functions to calculate the probability of exact recovery for $n=100$ and alpha, beta both varying between 0 and 5. Make a 3-d plot showing the probability of exact recovery as a function of alpha and beta.</font>

In [6]:
# sample code for 3D plots - please tweak it to plot the probability of exact recovery
#from IPython.core.display import clear_output
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np


n = 10
limit = 5 
 
alpha = np.linspace(0,limit,limit*10 +1)
beta = np.linspace(0,limit,limit*10 +1)
prob=np.zeros((len(alpha) ,len(beta) ))
for a in alpha:
    for b in beta:
        prob[a,b] = recon_prob_sdp(n, a, b)
        print a,b,"done"
#        clear_output
#prob = recon_prob_sdp(10,1,1)        
#recon_prob_sdp(n, 1, 1)  
fig = plt.figure()
ax = fig.gca(projection='3d')
X = alpha
Y = beta
X, Y = np.meshgrid(X, Y)
#R = np.sqrt(X**2 + Y**2)
Z = prob #np.sin(R)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(-1.01, 1.01)
ax.set_xlabel('Alpha')
ax.set_ylabel('Beta')
ax.set_zlabel('Probability of Recovery')
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()


0.0 0.0 done
0.0 0.1 done
0.0 0.2 done
0.0 0.3 done
0.0 0.4 done


C:\Users\rock-\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:18: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


KeyboardInterrupt: 

In [ ]:
print prob[prob!=0]

----

## Problem 2: Spectral Methods for Community Detection (40 pts)

In spectral methods, a network is first represented by an  $n\times n$ matrix $L$ (called Laplacian matrix) defined as 
$$
L_{ij}=\begin{cases}
d_i & \text{if } i = j \\
-1 & \text{if }(i,j)\in E\\
0 & \text{if }(i,j)\notin E,
\end{cases}
$$
where $d_i$ is the degree of the network. The smallest eigenvalue of $L$ is equal to zero because $L$ times the all 1s vector is equal to zero. The next $k-1$ smallest eigenvalues usually determine the number of clusters. In other words, if you can find $k-1$ eigenvalues that are very close to zero, then the graph has $k$ commnunities. If the graph has 2 communities, the eigenvector corresponding to the small non-zero eigenvalue can be used to figure out which nodes belong to community A and which ones belong to community B.

### <font color=blue>Modify the following function to create the Laplacian matrix of a randomly generate SBM graph.</font>

In [7]:
def generate_sbm(n, alpha, beta):
    """
    Generate the A matrix for an SBM.
    inputs:  n: total number of nodes, 
             alpha: parameter alpha corresponding to the in-cluster connection probability
             beta: parameter beta corresponding to the cross-cluster connection probability
    outputs: A: the "A" matrix for the SBM. A(i,i)=0 for all i; A(i,j) = 1 if (i,j) is an edge; A(i,j)=-1 otherwise.
             truth: the ground truth of the two clusters, represented with +/- 1
    both A and truth are in the CVX matrix data structure 
    """
    assert(n % 2 == 0)
    mid = int(n/2)
    # generate parameters
    p = alpha*np.log(n)/n
    if p > 1.0:
        p = 1.0
    q = beta*np.log(n)/n
    if q > 1.0:
        q = 1.0
    # generate A matrix
    A = np.zeros([n, n])
    A[0:mid, mid:n] = np.random.binomial(1, q, (mid, mid))
    for i in range(mid):
        for j in range(i+1, mid):
            A[i, j] = np.random.binomial(1, p)
    for i in range(mid, n):
        for j in range(i+1, n):
            A[i, j] = np.random.binomial(1, p)
    A = A+np.transpose(A)
    A = (A-0.5)*2
    for i in range(n):
        A[i, i] = 0
    # randomly permute the rows and columns
    perm = np.random.permutation(n)
    A = A[:, perm]
    A = A[perm, :]
    # find the ground truth
    argperm = np.argsort(perm)
    truth = np.zeros([n, 1])
    truth[argperm[0:mid], 0] = 1
    truth[argperm[mid:n], 0] = -1
    #Calculate the Laplacian
#    print A
    D = np.zeros([n, n])
    for i in xrange(0,n):
        D[i,i] = (A[i,] == 1).sum()
    Adj = np.copy(A)
    Adj[Adj == -1] = 0
#    print Adj
    L = D - Adj
    
    
    # return A and truth
    return L, matrix(A), matrix(truth)

def recon_prob_sdp(n, alpha, beta):
    """
    Find the probability of successful reconstruction given the parameters
    inputs:  n: total number of nodes, 
             alpha: parameter alpha corresponding to the in-cluster connection probability
             beta: parameter beta corresponding to the cross-cluster connection probability
    outputs: the simulated probability of successful reconstruction
    """
    assert(n % 2 == 0)
    num_tests = 1
    num_success = 0.0
    for t in range(num_tests):
        result = generate_sbm(n, alpha, beta)
        L = result[0]
        A = np.array(result[1])
        truth = np.array(result[2])
        eigVal = np.linalg.eigh(L)[0]
#        print eigVal[1]
        eigVec = np.linalg.eigh(L)[1][:,1] #store the eigenvector correspondong to the second smallest eiganvalue
        print eigVec
        eigVec[eigVec >= 0] = 1.0  #anything greater than 0 is in a community
        eigVec[eigVec < 0] = -1.0 #anything less than 0 is in the other community
        eigVec = eigVec.reshape(n,1) #reshape to make it a column vector
        print truth
        #        print sum(eigVec),sum(truth)
        if np.array_equal(eigVec,truth) or np.array_equal(eigVec * -1,truth): #check how many times we match the two correctly
            num_success = num_success + 1
    return float(num_success)/float(num_tests)

### <font color=blue>Use the above modified helper function to calculate the probability of exact recovery for $n=100$ and alpha, beta both varying between 0 and 5. Make a 3-d plot showing the probability of exact recovery as a function of alpha and beta.</font>

In [ ]:
#test = generate_sbm(6,1,1)
#print test[2]
n = 100 #100 nodes
limit = 5 #5 alpha/beta

#initialize the arrays
alpha = np.linspace(0,limit,limit*10 +1) 
beta = np.linspace(0,limit,limit*10 +1)
prob=np.zeros((len(alpha) ,len(beta)))
#compute probs
for a in alpha:
    for b in beta:
        prob[a,b] = recon_prob_sdp(n, a, b)
#print prob

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.

In [ ]:
from IPython.core.display import clear_output
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np

#plot them
fig = plt.figure()
ax = fig.gca(projection='3d')
X = alpha
Y = beta
X, Y = np.meshgrid(X, Y)
#R = np.sqrt(X**2 + Y**2)
Z = prob #np.sin(R)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(-1.01, 1.01)
ax.set_xlabel('Alpha')
ax.set_ylabel('Beta')
ax.set_zlabel('Probability of Recovery')
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

print prob[prob!=0]